In [89]:
import pandas as pd

## 전체 train sample
df = pd.read_csv('../train.csv',index_col=0)
set_level_list_3 = list(set(list(df['cat3'])))
set_level_list_3.sort()            
label_2_number = {label:i for i,label in enumerate(set_level_list_3)}
number_2_label = {i:label for i,label in enumerate(set_level_list_3)}

In [90]:
## 베스트 모델 추론할때 soft 보팅의 웨이트값을 찾기 위함
result_df = pd.DataFrame(list(label_2_number.keys()),columns=['index'])

target_df = pd.read_csv('./data_save_temp/fold_3_epoch_15_valdf_rank_0.csv')

target_df = target_df[['id','cat3','predict_label_number','target','predict_correct']]
df1 = target_df['cat3'].value_counts().to_frame(name='val_전체수').reset_index()
df2 = target_df.groupby(['cat3'])['predict_correct'].sum().to_frame(name='val_정답수').reset_index()
result_df = pd.merge(left = result_df , right = df1, left_on = "index",right_on = "index", how = "left")
result_df


,index,val_전체수
0,5일장,18.0
1,ATV,1.0
2,MTB,NaN
3,강,10.0
4,게스트하우스,11.0
...,...,...
123,헬스투어,2.0
124,헹글라이딩/패러글라이딩,1.0
125,호수,4.0
126,홈스테이,8.0


In [91]:
result_df = pd.merge(left = result_df , right = df2, left_on = "index",right_on = "cat3", how = "left").drop('cat3',1)

result_df['val_틀린수'] = result_df['val_전체수'] - result_df['val_정답수']
result_df['acc'] = result_df['val_정답수'] / result_df['val_전체수']
result_df = result_df.fillna(0)
result_df
# result_df.to_csv('./temp.csv')


,index,val_전체수,val_정답수,val_틀린수,acc
0,5일장,18.0,16.0,2.0,0.888889
1,ATV,1.0,1.0,0.0,1.000000
2,MTB,0.0,0.0,0.0,0.000000
3,강,10.0,5.0,5.0,0.500000
4,게스트하우스,11.0,10.0,1.0,0.909091
...,...,...,...,...,...
123,헬스투어,2.0,1.0,1.0,0.500000
124,헹글라이딩/패러글라이딩,1.0,0.0,1.0,0.000000
125,호수,4.0,0.0,4.0,0.000000
126,홈스테이,8.0,2.0,6.0,0.250000


In [92]:
weight_dict = { index:acc for index,acc in zip(result_df['index'],result_df['acc'])}
# weight_dict

In [93]:
target_df = pd.read_csv('./data_save_temp/fold_3_epoch_15_valdf_rank_0.csv')
temp = target_df.iloc[0]['predict_logit'].replace('[','')
temp = temp.replace(']','')
temp = temp.replace("'",'')
temp = temp.replace(" ",'')
temp = temp.split(',')
print(temp,type(temp))

['0.2741032838821411', '0.726055383682251', '3.8537850379943848', '-0.5932121872901917', '0.5228717923164368', '-0.49769100546836853', '0.6942906379699707', '0.7193548083305359', '1.2125492095947266', '0.8325292468070984', '0.6441237330436707', '-1.050925850868225', '-0.5559065937995911', '2.431729316711426', '-0.18206194043159485', '0.5282250046730042', '-0.27948659658432007', '-1.399703860282898', '-1.368259072303772', '0.13256937265396118', '-0.7375528216362', '0.2824725806713104', '3.5026369094848633', '1.7273540496826172', '0.11742763221263885', '1.792862892150879', '1.7945923805236816', '1.3292771577835083', '3.0702829360961914', '0.40677037835121155', '3.835261821746826', '-0.6626306176185608', '-0.06368841230869293', '1.9272936582565308', '0.4543682932853699', '-0.0673721507191658', '2.7410759925842285', '1.0621751546859741', '1.1277921199798584', '0.9499882459640503', '0.7908570766448975', '0.40709197521209717', '0.5867888927459717', '2.76792311668396', '-1.9058034420013428', 

In [94]:
import numpy as np
temp2 = []
for i in temp:
    temp2.append(float(i))
    
print(np.max(temp2),np.argmax(temp2))

14.320274353027344 103


In [97]:
print(number_2_label[103],target_df.iloc[0])

'카지노'